In [1]:
import torch
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import numpy as np
from matplotlib import pyplot as plt
import resnet_frelu as resnet
import os
from main import AverageMeter, ProgressMeter, accuracy, train, validate
import time

<h2>Validate current set of weights</h2>

In [2]:
model = torch.load('frelu_resnet50.pth')

In [2]:
data = 'C://ImageNet/'
traindir = os.path.join(data, 'train')
valdir = os.path.join(data, 'val')
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])

total_steps = 300000
learning_rate = 0.001

criterion = torch.nn.CrossEntropyLoss().cuda()

train_dataset = datasets.ImageFolder(traindir,
                                     transforms.Compose([
                                         transforms.RandomResizedCrop(224),
                                         transforms.RandomHorizontalFlip(),
                                         transforms.ColorJitter(brightness=0.4, contrast=0.4, saturation=0.4),
                                         transforms.ToTensor(),
                                         normalize
                                        ]))

In [4]:
val_loader = torch.utils.data.DataLoader(
                                datasets.ImageFolder(valdir, transforms.Compose([
                                    transforms.Resize(256),
                                    transforms.CenterCrop(224),
                                    transforms.ToTensor(),
                                    normalize
                                ])),
                                batch_size=100, shuffle=True,
                                num_workers=4, pin_memory=True)

In [5]:
model.cuda()
validate(val_loader, model, criterion, {})

Test: [  0/500]	Time  5.682 ( 5.682)	Loss 7.2411e-01 (7.2411e-01)	Acc@1  85.00 ( 85.00)	Acc@5  97.00 ( 97.00)
Test: [100/500]	Time  0.553 ( 0.602)	Loss 8.2409e-01 (9.4904e-01)	Acc@1  78.00 ( 77.50)	Acc@5  95.00 ( 93.64)
Test: [200/500]	Time  0.559 ( 0.579)	Loss 7.7373e-01 (9.3647e-01)	Acc@1  84.00 ( 77.70)	Acc@5  94.00 ( 93.80)
Test: [300/500]	Time  0.555 ( 0.573)	Loss 1.3577e+00 (9.4489e-01)	Acc@1  74.00 ( 77.42)	Acc@5  90.00 ( 93.77)
Test: [400/500]	Time  0.559 ( 0.571)	Loss 9.4712e-01 (9.3480e-01)	Acc@1  81.00 ( 77.57)	Acc@5  95.00 ( 93.87)
 * Acc@1 77.606 Acc@5 93.836


tensor(77.6060, device='cuda:0')

<h2>Create and train new ResNet with FReLU activations (primitive example)</h2>

In [3]:
model = resnet.resnet101()

In [ ]:
model

In [5]:
optimizer = torch.optim.SGD(
        model.parameters(),
        lr=learning_rate / 10,
        momentum=0.9,
        weight_decay=1e-4,
    )

In [6]:
model.train()
train_sampler = None
train_loader = torch.utils.data.DataLoader(
        train_dataset, batch_size=4, shuffle=(train_sampler is None),
        num_workers=1, pin_memory=True, sampler=train_sampler)

In [ ]:
for e in range(10):
    train(train_loader, model, criterion, optimizer, e)
    torch.save(model, 'FResNet50_' + str(e) + '.pth')